# Cleaning Data for the random sample dataset

In [1]:
import pandas as pd

In [4]:
pd.set_option("display.max_columns", None)

In [22]:
df = pd.read_csv("data/flights_random_sample_400000.csv")

In [23]:
df.head()

,fl_date,mkt_unique_carrier,branded_code_share,mkt_carrier,mkt_carrier_fl_num,op_unique_carrier,tail_num,op_carrier_fl_num,origin_airport_id,origin,origin_city_name,dest_airport_id,dest,dest_city_name,crs_dep_time,dep_time,dep_delay,taxi_out,wheels_off,wheels_on,taxi_in,crs_arr_time,arr_time,arr_delay,cancelled,cancellation_code,diverted,dup,crs_elapsed_time,actual_elapsed_time,air_time,flights,distance,carrier_delay,weather_delay,nas_delay,security_delay,late_aircraft_delay,first_dep_time,total_add_gtime,longest_add_gtime,no_name
0,2018-03-07,AA,AA,AA,311,AA,N906NN,311,13303,MIA,"Miami, FL",10529,BDL,"Hartford, CT",1940,NaN,NaN,NaN,NaN,NaN,NaN,2249,NaN,NaN,1.0,B,0.0,N,189.0,NaN,NaN,1.0,1194.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2018-03-07,AA,AA,AA,332,AA,N946NN,332,12953,LGA,"New York, NY",13930,ORD,"Chicago, IL",1330,NaN,NaN,NaN,NaN,NaN,NaN,1512,NaN,NaN,1.0,B,0.0,N,162.0,NaN,NaN,1.0,733.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2018-03-07,AA,AA,AA,465,AA,N200UU,465,14107,PHX,"Phoenix, AZ",14679,SAN,"San Diego, CA",835,833.0,-2.0,13.0,846.0,838.0,2.0,851,840.0,-11.0,0.0,NaN,0.0,N,76.0,67.0,52.0,1.0,304.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2018-03-07,AA,AA,AA,550,AA,N947UW,550,14100,PHL,"Philadelphia, PA",10529,BDL,"Hartford, CT",2050,NaN,NaN,NaN,NaN,NaN,NaN,2156,NaN,NaN,1.0,B,0.0,N,66.0,NaN,NaN,1.0,196.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2018-03-07,AA,AA,AA,591,AA,N833AW,591,11057,CLT,"Charlotte, NC",11278,DCA,"Washington, DC",1431,1537.0,66.0,16.0,1553.0,1648.0,3.0,1559,1651.0,52.0,0.0,NaN,0.0,N,88.0,74.0,55.0,1.0,331.0,52.0,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN


### Fixing null values and dropping redundant/irrelevant columns

In [77]:
# mkt_carrier is the same as mke_unique_carrier
df = df.drop(["mkt_unique_carrier"], axis=1)

In [32]:
# Drop rows that don't have an arrival time (can't predict arr_delay if play never arrived)
df = df.dropna(subset=["arr_time"])

In [44]:
# drop flights that were diverted (they don't have an arr_time)
df = df[df["diverted"].ne(1)]

In [51]:
# The remaining flights with null values in arr_delay have the same predicted arrival time as actual arrival time
df["arr_delay"].fillna(0, inplace=True)

In [61]:
# The remaining flights with null values in dep_delay have the same predicted departure time as actual departure time
df["dep_delay"].fillna(0, inplace=True)

In [55]:
# drop flights that weren't actually flights??? These all seem to be codeshares from hawaiian airlines.
df = df.dropna(subset=["air_time"])

In [73]:
# Our dataframe no longer has any cancelled flights in it
df = df.drop(["cancelled", "cancellation_code"], axis=1)

In [80]:
# no_name is an empty column
df = df.drop(["no_name"], axis=1)

In [90]:
# create dataframe for delay details (may or may not need this later. probably not)
delay_details_cols = list(df.columns[-8:])
df_delay_details = df[delay_details_cols]
df_delay_details.head()

In [90]:
# drop delay details columns
df = df.drop(delay_details_cols, axis=1)

In [102]:
# dataframe no longer contains any flights that were diverted
df = df.drop(["diverted"], axis=1)

In [138]:
# no unique values in this column (all "N")
df = df.drop(["dup"], axis=1)

In [147]:
# no unique values in this column (all "1")
df = df.drop(["flights"], axis=1)

In [148]:
df.head()

,fl_date,branded_code_share,mkt_carrier,mkt_carrier_fl_num,op_unique_carrier,tail_num,op_carrier_fl_num,origin_airport_id,origin,origin_city_name,dest_airport_id,dest,dest_city_name,crs_dep_time,dep_time,dep_delay,taxi_out,wheels_off,wheels_on,taxi_in,crs_arr_time,arr_time,arr_delay,crs_elapsed_time,actual_elapsed_time,air_time,distance
2,2018-03-07,AA,AA,465,AA,N200UU,465,14107,PHX,"Phoenix, AZ",14679,SAN,"San Diego, CA",835,833.0,-2.0,13.0,846.0,838.0,2.0,851,840.0,-11.0,76.0,67.0,52.0,304.0
4,2018-03-07,AA,AA,591,AA,N833AW,591,11057,CLT,"Charlotte, NC",11278,DCA,"Washington, DC",1431,1537.0,66.0,16.0,1553.0,1648.0,3.0,1559,1651.0,52.0,88.0,74.0,55.0,331.0
5,2018-03-07,AA,AA,600,AA,N151UW,600,11697,FLL,"Fort Lauderdale, FL",11057,CLT,"Charlotte, NC",603,557.0,-6.0,18.0,615.0,746.0,19.0,809,805.0,-4.0,126.0,128.0,91.0,632.0
6,2018-03-07,AA,AA,1805,AA,N924US,1805,11057,CLT,"Charlotte, NC",10721,BOS,"Boston, MA",1135,1129.0,-6.0,11.0,1140.0,1312.0,12.0,1352,1324.0,-28.0,137.0,115.0,92.0,728.0
7,2018-03-07,AA,AA,2615,AA,N945NN,2615,11057,CLT,"Charlotte, NC",15370,TUL,"Tulsa, OK",1820,1812.0,-8.0,11.0,1823.0,1936.0,6.0,2002,1942.0,-20.0,162.0,150.0,133.0,842.0


In [106]:
df.shape

(390060, 29)

In [107]:
df.isnull().sum()

fl_date                0
branded_code_share     0
mkt_carrier            0
mkt_carrier_fl_num     0
op_unique_carrier      0
tail_num               0
op_carrier_fl_num      0
origin_airport_id      0
origin                 0
origin_city_name       0
dest_airport_id        0
dest                   0
dest_city_name         0
crs_dep_time           0
dep_time               0
dep_delay              0
taxi_out               0
wheels_off             0
wheels_on              0
taxi_in                0
crs_arr_time           0
arr_time               0
arr_delay              0
dup                    0
crs_elapsed_time       0
actual_elapsed_time    0
air_time               0
flights                0
distance               0
dtype: int64

### further analysis of outliers / incorrect data

In [149]:
df.describe()

,mkt_carrier_fl_num,op_carrier_fl_num,origin_airport_id,dest_airport_id,crs_dep_time,dep_time,dep_delay,taxi_out,wheels_off,wheels_on,taxi_in,crs_arr_time,arr_time,arr_delay,crs_elapsed_time,actual_elapsed_time,air_time,distance
count,390058.000000,390058.000000,390058.000000,390058.000000,390058.000000,390058.000000,390058.000000,390058.000000,390058.000000,390058.000000,390058.000000,390058.000000,390058.000000,390058.000000,390058.000000,390058.000000,390058.000000,390058.000000
mean,2718.410926,2718.345302,12676.967948,12681.838778,1328.864210,1333.971335,10.491965,17.511406,1358.154159,1458.801278,7.684683,1482.925544,1463.658176,5.499105,138.100862,133.113665,107.917576,770.283576
std,1859.335215,1859.396220,1526.030261,1522.948134,490.482159,504.579315,47.911094,10.036387,506.093594,532.940981,6.134376,517.171786,537.038187,50.114987,71.878009,71.718236,69.804743,587.824801
min,1.000000,1.000000,10135.000000,10135.000000,1.000000,1.000000,-45.000000,0.000000,1.000000,1.000000,0.000000,1.000000,1.000000,-85.000000,20.000000,16.000000,8.000000,31.000000
25%,1118.000000,1118.000000,11292.000000,11292.000000,914.000000,915.000000,-6.000000,11.000000,931.000000,1043.000000,4.000000,1100.000000,1048.000000,-15.000000,87.000000,82.000000,58.000000,341.000000
50%,2297.000000,2297.000000,12889.000000,12889.000000,1320.000000,1327.000000,-2.000000,15.000000,1340.000000,1458.000000,6.000000,1510.000000,1502.000000,-6.000000,119.000000,114.000000,89.000000,604.000000
75%,4295.000000,4294.000000,14057.000000,14057.000000,1735.000000,1744.000000,7.000000,20.000000,1800.000000,1909.000000,9.000000,1916.000000,1914.000000,8.000000,168.000000,163.000000,137.000000,1005.000000
max,9392.000000,9392.000000,16869.000000,16869.000000,2359.000000,2400.000000,1855.000000,165.000000,2400.000000,2400.000000,221.000000,2400.000000,2400.000000,1863.000000,703.000000,715.000000,688.000000,5095.000000


In [125]:
# Drop flights that had a negative elapsed time for some reason
df = df[df["crs_elapsed_time"].gt(0)]

In [132]:
# fixing a single flight value
df.loc[326616, "crs_elapsed_time"] = 89.0

/Users/zacharyargentin/opt/anaconda3/envs/bootcamp/lib/python3.8/site-packages/pandas/core/indexing.py:1817: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(loc, value, pi)


In [150]:
df.sort_values("taxi_in").tail(10)

,fl_date,branded_code_share,mkt_carrier,mkt_carrier_fl_num,op_unique_carrier,tail_num,op_carrier_fl_num,origin_airport_id,origin,origin_city_name,dest_airport_id,dest,dest_city_name,crs_dep_time,dep_time,dep_delay,taxi_out,wheels_off,wheels_on,taxi_in,crs_arr_time,arr_time,arr_delay,crs_elapsed_time,actual_elapsed_time,air_time,distance
311669,2019-07-31,DL,DL,2915,DL,N184DN,2915,14869,SLC,"Salt Lake City, UT",10721,BOS,"Boston, MA",935,933.0,-2.0,20.0,953.0,1705.0,151.0,1626,1936.0,190.0,291.0,483.0,312.0,2105.0
313654,2018-01-05,DL,DL,984,DL,N352NW,984,10397,ATL,"Atlanta, GA",12478,JFK,"New York, NY",2142,2158.0,16.0,8.0,2206.0,2344.0,151.0,5,215.0,130.0,143.0,257.0,98.0,760.0
240341,2018-07-27,DL,DL,1669,DL,N6715C,1669,10397,ATL,"Atlanta, GA",13204,MCO,"Orlando, FL",1815,1828.0,13.0,17.0,1845.0,1941.0,157.0,2000,2218.0,138.0,105.0,230.0,56.0,404.0
369409,2018-11-15,DL,DL,1841,DL,N834MH,1841,12892,LAX,"Los Angeles, CA",12478,JFK,"New York, NY",1420,1419.0,-1.0,26.0,1445.0,2235.0,165.0,2251,120.0,149.0,331.0,481.0,290.0,2475.0
336655,2018-08-18,AA,AA,2720,AA,N4005X,2720,15919,XNA,"Fayetteville, AR",11298,DFW,"Dallas/Fort Worth, TX",1816,2014.0,118.0,136.0,2230.0,2314.0,169.0,1942,203.0,381.0,86.0,349.0,44.0,280.0
18137,2019-08-08,DL,DL,2629,DL,N321DH,2629,10397,ATL,"Atlanta, GA",12953,LGA,"New York, NY",1430,1543.0,73.0,24.0,1607.0,1757.0,171.0,1645,2048.0,243.0,135.0,305.0,110.0,762.0
240169,2018-07-27,NK,NK,423,NK,N623NK,423,10721,BOS,"Boston, MA",13204,MCO,"Orlando, FL",1550,1605.0,15.0,14.0,1619.0,1929.0,171.0,1910,2220.0,190.0,200.0,375.0,190.0,1121.0
336797,2018-08-18,AA,AA,2372,AA,N163AA,2372,14683,SAT,"San Antonio, TX",11298,DFW,"Dallas/Fort Worth, TX",1950,2027.0,37.0,134.0,2241.0,2325.0,172.0,2104,217.0,313.0,74.0,350.0,44.0,247.0
336625,2018-08-18,AA,AA,1399,AA,N955NN,1399,12892,LAX,"Los Angeles, CA",11298,DFW,"Dallas/Fort Worth, TX",1820,1819.0,-1.0,22.0,1841.0,2316.0,176.0,2330,212.0,162.0,190.0,353.0,155.0,1235.0
233560,2018-07-16,DL_CODESHARE,DL,3322,9E,N197PQ,3322,10397,ATL,"Atlanta, GA",11973,GPT,"Gulfport/Biloxi, MS",1247,1301.0,14.0,18.0,1319.0,1317.0,221.0,1312,1658.0,226.0,85.0,297.0,58.0,352.0


### dtypes

In [156]:
df2 = df.copy()

In [157]:
df2.dtypes

fl_date                 object
branded_code_share      object
mkt_carrier             object
mkt_carrier_fl_num       int64
op_unique_carrier       object
tail_num                object
op_carrier_fl_num        int64
origin_airport_id        int64
origin                  object
origin_city_name        object
dest_airport_id          int64
dest                    object
dest_city_name          object
crs_dep_time             int64
dep_time               float64
dep_delay              float64
taxi_out               float64
wheels_off             float64
wheels_on              float64
taxi_in                float64
crs_arr_time             int64
arr_time               float64
arr_delay              float64
crs_elapsed_time       float64
actual_elapsed_time    float64
air_time               float64
distance               float64
dtype: object

In [158]:
help(df2.astype)

Help on method astype in module pandas.core.generic:

astype(dtype, copy: 'bool_t' = True, errors: 'str' = 'raise') -> 'FrameOrSeries' method of pandas.core.frame.DataFrame instance
    Cast a pandas object to a specified dtype ``dtype``.
    
    Parameters
    ----------
    dtype : data type, or dict of column name -> data type
        Use a numpy.dtype or Python type to cast entire pandas object to
        the same type. Alternatively, use {col: dtype, ...}, where col is a
        column label and dtype is a numpy.dtype or Python type to cast one
        or more of the DataFrame's columns to column-specific types.
    copy : bool, default True
        Return a copy when ``copy=True`` (be very careful setting
        ``copy=False`` as changes to values then may propagate to other
        pandas objects).
    errors : {'raise', 'ignore'}, default 'raise'
        Control raising of exceptions on invalid data for provided dtype.
    
        - ``raise`` : allow exceptions to be raised


In [172]:
# change numerical 'id' datatypes to object
df2 = df2.astype({"mkt_carrier_fl_num": "object",
            "op_carrier_fl_num": "object",
            "origin_airport_id": "object",
            "dest_airport_id": "object"})

In [ ]:
# change object to string datastypes


In [173]:
df2.dtypes

fl_date                 object
branded_code_share      object
mkt_carrier             object
mkt_carrier_fl_num      object
op_unique_carrier       object
tail_num                object
op_carrier_fl_num       object
origin_airport_id       object
origin                  object
origin_city_name        object
dest_airport_id         object
dest                    object
dest_city_name          object
crs_dep_time             int64
dep_time               float64
dep_delay              float64
taxi_out               float64
wheels_off             float64
wheels_on              float64
taxi_in                float64
crs_arr_time             int64
arr_time               float64
arr_delay              float64
crs_elapsed_time       float64
actual_elapsed_time    float64
air_time               float64
distance               float64
dtype: object